In [3]:
import json
import random 
data_path = "/home/team_nlp/quangminh/test_fc/test_fc/data_test_fc/results.jsonl"

with open(data_path, 'r') as f:
    data = [json.loads(line) for line in f]

random_sample = random.sample(data, 5)
for sample in random_sample:
    print(f"FC result for input: {sample['query']}")
    mutual_keys = set(sample['fc_payload'].keys()).intersection(set(sample['gpt_payload'].keys()))
    for key in mutual_keys: 
        print(f"Key: {key}, GPT: {sample['gpt_payload'][key]}, CMC model: {sample['fc_payload'][key]}")
    print("---------------------------------------------------")

FC result for input: Bạn nghĩ gì về trí tuệ nhân tạo?
Key: loai_van_ban, GPT: [], CMC model: 
Key: tinh_trang_hieu_luc, GPT: [], CMC model: Còn hiệu lực,Hết hiệu lực một phần
Key: use_rag, GPT: False, CMC model: False
Key: ngay_co_hieu_luc_end, GPT: , CMC model: 
Key: use_memory, GPT: False, CMC model: False
Key: ngay_co_hieu_luc_start, GPT: , CMC model: 
Key: query, GPT: , CMC model: Bạn nghĩ gì về trí tuệ nhân tạo?
Key: co_quan_ban_hanh, GPT: , CMC model: 
Key: ten_van_ban, GPT: , CMC model: 
Key: ngay_ban_hanh_start, GPT: , CMC model: 
Key: ngay_ban_hanh_end, GPT: , CMC model: 
Key: reference_text, GPT: , CMC model: 
Key: intent, GPT: general, CMC model: chitchat
Key: retrieve_level, GPT: document, CMC model: document
Key: loai_hinh_van_ban, GPT: văn bản quy phạm, CMC model: 
Key: need_subquery, GPT: False, CMC model: False
Key: query_external_search, GPT: , CMC model: 
---------------------------------------------------
FC result for input: Pháp luật Việt Nam có quy định về bảo vệ 